In [72]:
import numpy as np
samples = np.load('data/global_meteo_ex.npy')
n_subsample = 800000
subsample_indices = np.random.choice(samples.shape[1], size=n_subsample, replace=False)
# Extract features and target
X = samples[0:2,subsample_indices].T
y = samples[2,subsample_indices]

In [73]:
stats_dict = np.load('data/stats_300_meteo_ex.npy', allow_pickle=True).item()
ssrd_mean = stats_dict['ssrd_mean']
ssrd_std = stats_dict['ssrd_std']
vpd_mean = stats_dict['vpd_mean']
vpd_std = stats_dict['vpd_std']
gpp_mean = stats_dict['gpp_mean']
gpp_std = stats_dict['gpp_std']

In [74]:
# Normalize your input features X
X_normalized = np.zeros_like(X)
X_normalized[:, 0] = (X[:, 0] - ssrd_mean) / ssrd_std  # SSRD
X_normalized[:, 1] = (X[:, 1] - vpd_mean) / vpd_std    # VPD    

In [75]:
import torch
X_tensor = torch.tensor(X_normalized, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(1)  # Ensure correct shape

In [76]:
from model import ClimateDataset
from torch.utils.data import DataLoader
# Create DataLoaders
batch_size = 1024  # You can adjust this value based on your available memory and GPU capacity
loader = DataLoader(ClimateDataset(X_tensor, y_tensor), batch_size=batch_size, shuffle=False)

In [77]:
from model import NeuralNet
from torch import nn, optim
# Initialize model, loss function, and optimizer
input_dim = 2  # Number of predictors (ssrd and vpd)
model = NeuralNet(input_dim)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained weights
model.load_state_dict(torch.load('./outputs/model_weights_default_meteo_ex.pth', map_location=device))
model.to(device)
model.eval()  # Set to evaluation mode

print(f"Model loaded on device: {device}")
print("Pre-trained weights loaded successfully!")

Model loaded on device: cuda
Pre-trained weights loaded successfully!


In [78]:
# Generate predictions using the loaded model
predictions = []
actual_values = []

print("Generating predictions...")
with torch.no_grad():
    for batch_X, batch_y in loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        outputs = model(batch_X)
        
        # Store predictions and actual values
        predictions.append(outputs.cpu().numpy())
        actual_values.append(batch_y.cpu().numpy())

# Concatenate all predictions and actual values
predictions = np.concatenate(predictions, axis=0).flatten()
actual_values = np.concatenate(actual_values, axis=0).flatten()

Generating predictions...


In [79]:
predictions = predictions * gpp_std + gpp_mean  # Denormalize predictions

In [80]:
print(f"Generated {len(predictions):,} predictions")
print(f"Prediction range: [{predictions.min():.6f}, {predictions.max():.6f}]")
print(f"Actual range: [{actual_values.min():.6f}, {actual_values.max():.6f}]")

# Calculate some basic metrics
mse = np.mean((predictions - actual_values) ** 2)
rmse = np.sqrt(mse)
mae = np.mean(np.abs(predictions - actual_values))

print(f"\nModel Performance:")
print(f"MSE: {mse:.10f}")
print(f"RMSE: {rmse:.10f}")
print(f"MAE: {mae:.10f}")

# Save the predictions and actual values
np.save('./outputs/predictions_default_meteo_ex.npy', predictions)

Generated 800,000 predictions
Prediction range: [-17.172349, 426.811300]
Actual range: [-0.000420, 501.899658]

Model Performance:
MSE: 140.5843301360
RMSE: 11.8568263096
MAE: 7.0173517870


In [90]:
output = np.concatenate((X, predictions.reshape(-1, 1), actual_values.reshape(-1, 1)), axis=1)
output.shape

(800000, 4)

In [91]:
np.save('./outputs/default_forward_meteo_ex.npy', output)